This GUI will help in annotating images by creating a txt file with the coordinates of the bounding box as well as the labels of the bounding boxes

In [1]:
import tkinter as tk
from tkinter import simpledialog, filedialog
from PIL import Image, ImageTk
import cv2
import os
import csv
import tkinter.ttk as ttk
import threading
import random

In [2]:
class AnnotationTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Annotation Tool")

        # Frame for buttons
        button_frame = tk.Frame(root, bg='gray')
        button_frame.pack(side="bottom", fill="x")

        # Styling buttons
        button_style = {'bg': 'lightblue', 'fg': 'black'}

        # Confirm, Delete, Reset buttons
        self.confirm_button = tk.Button(button_frame, text="Confirm", command=self.confirm_annotations, **button_style)
        self.confirm_button.pack(side="left")
        self.delete_button = tk.Button(button_frame, text="Delete", command=self.delete_current_image, **button_style)
        self.delete_button.pack(side="left")
        self.reset_button = tk.Button(button_frame, text="Reset Box", command=self.reset_current_box, **button_style)
        self.reset_button.pack(side="left")

        # Set fixed size and background for the canvas
        self.canvas = tk.Canvas(root, width=1200, height=900, bg='black', cursor="cross")
        self.canvas.pack()

        # Rest of the initialization code
        self.image_dir = filedialog.askdirectory(title="Select Image Directory")
        self.images = []
        self.current_image_index = 0
        self.current_image = None
        self.start_x = None
        self.start_y = None
        self.rect = None
        self.rect_label = None
        self.annotations = []
        self.load_images()
        self.display_next_image()
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)

    def reset_current_box(self):
        # Delete the current rectangle and reset start coordinates
        if self.rect:
            self.canvas.delete(self.rect)
            self.rect = None
            self.start_x = None
            self.start_y = None

    def load_images(self):
        self.images = [os.path.join(self.image_dir, f) for f in os.listdir(self.image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def display_next_image(self):
        canvas_width = 1200
        canvas_height = 1000

        if self.current_image_index < len(self.images):
            self.canvas.delete("all")  # Clear the previous image and box

            image_path = self.images[self.current_image_index]
            original_image = Image.open(image_path)

            # Calculate the new size maintaining aspect ratio
            ratio = min(canvas_width / original_image.width, canvas_height / original_image.height)
            new_width = int(original_image.width * ratio)
            new_height = int(original_image.height * ratio)

            # Resize the image for display
            display_image = original_image.resize((new_width, new_height), Image.ANTIALIAS)
            self.current_image = ImageTk.PhotoImage(display_image)

            # Calculate position to center the image
            x = (canvas_width - new_width) // 2
            y = (canvas_height - new_height) // 2
            self.canvas.create_image(x, y, anchor='nw', image=self.current_image)
            self.current_image_index += 1
        else:
            print("No more images.")
            self.root.quit()
    
    def delete_current_image(self):
        if self.current_image_index > 0:
            current_image_path = self.images[self.current_image_index - 1]
            try:
                os.remove(current_image_path)  # Delete the current image file
                print(f"Deleted: {current_image_path}")
            except OSError as e:
                print(f"Error: {e.strerror}")
            
            # Remove the deleted image from the list and adjust the index
            del self.images[self.current_image_index - 1]
            self.current_image_index -= 1
        
        self.display_next_image()  # Display the next image

    def on_button_press(self, event):
        # Start drawing the bounding box
        self.start_x = event.x
        self.start_y = event.y
        self.rect = self.canvas.create_rectangle(self.start_x, self.start_y, event.x, event.y, outline='red')
        # Bind the motion event to dynamically update the bounding box
        self.canvas.bind("<Motion>", self.on_mouse_move)

    def on_mouse_move(self, event):
        # Update the size of the rectangle as the mouse is moving
        self.canvas.coords(self.rect, self.start_x, self.start_y, event.x, event.y)
    
    def on_button_release(self, event):
        # Unbind the motion event
        self.canvas.unbind("<Motion>")

        # Finalize the size of the rectangle
        self.canvas.coords(self.rect, self.start_x, self.start_y, event.x, event.y)

        # Ask for label
        label = simpledialog.askstring("Label", "Enter the label:")
        if label:
            color = self.get_unique_color()
            self.canvas.itemconfig(self.rect, outline=color)

            # Create and store the label text element
            label_x = event.x + 10
            label_y = event.y - 20
            self.rect_label = self.canvas.create_text(label_x, label_y, text=label, anchor='nw', fill=color)

            # Calculate normalized coordinates
            x_center = (self.start_x + event.x) / 2
            y_center = (self.start_y + event.y) / 2
            width = abs(event.x - self.start_x)
            height = abs(event.y - self.start_y)

            # Append annotation to the list
            self.annotations.append({
                'image': self.images[self.current_image_index - 1],
                'bbox': [self.start_x, self.start_y, event.x, event.y],
                'label': label
            })
        else:
            # Delete the rectangle if labeling is canceled
            self.reset_current_box()

    def reset_current_box(self):
        # Check if there is a current rectangle
        if self.rect:
            # Delete the rectangle from the canvas
            self.canvas.delete(self.rect)
            self.rect = None

            # Also delete the associated label, if any
            if self.rect_label:
                self.canvas.delete(self.rect_label)
                self.rect_label = None

            # Remove the last added annotation for the current image
            # Assuming the last annotation added corresponds to the current box
            if self.annotations and self.annotations[-1]['image'] == self.images[self.current_image_index - 1]:
                self.annotations.pop()

    def get_unique_color(self):
        # Function to generate a unique color for each box
        # This can be as simple or complex as needed
        r = lambda: random.randint(0,255)
        return f'#{r():02x}{r():02x}{r():02x}'

    def save_annotations(self):
        label_dir = "labels"
        os.makedirs(label_dir, exist_ok=True)

        if self.current_image_index < len(self.images):
            image_path = self.images[self.current_image_index - 1]
            current_image_annotations = [ann for ann in self.annotations if ann['image'] == image_path]

            print(f"Saving annotations for: {image_path}")  # Debugging
            print(f"Annotations: {current_image_annotations}")  # Debugging

            with Image.open(image_path) as img:
                img_width, img_height = img.size

            base_name = os.path.basename(image_path)
            unique_identifier = base_name.split('.')[0]
            file_name = f"{unique_identifier}.txt"
            file_path = os.path.join(label_dir, file_name)
            print(f"Saving file: {file_path}")

            with open(file_path, 'w') as file:
                for ann in current_image_annotations:
                    # Normalize coordinates
                    x_center = (ann['bbox'][0] + ann['bbox'][2]) / 2 / img_width
                    y_center = (ann['bbox'][1] + ann['bbox'][3]) / 2 / img_height
                    width = abs(ann['bbox'][2] - ann['bbox'][0]) / img_width
                    height = abs(ann['bbox'][3] - ann['bbox'][1]) / img_height

                    # Use the label provided by the user for each bounding box
                    file.write(f"{ann['label']} {x_center} {y_center} {width} {height}\n")

    def confirm_annotations(self):
        self.save_annotations()

        # Clear annotations for the current image
        self.annotations = [ann for ann in self.annotations if ann['image'] != self.images[self.current_image_index - 1]]

        # Clear the canvas and reset for the next image
        self.canvas.delete("all")
        if self.current_image_index < len(self.images):
            self.display_next_image()
        else:
            print("No more images.")
            self.root.quit()

In [3]:
# Main function to start the GUI
def main():
    root = tk.Tk()
    # root.geometry("1000x800")  # Set the size of the window
    app = AnnotationTool(root)
    root.mainloop()
# # Run the main function in a separate thread
# thread = threading.Thread(target=main)
# thread.start()

In [4]:
if __name__ == "__main__":
    main()

C:\Users\Grego\AppData\Local\Temp\ipykernel_4580\3490588688.py:67: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  display_image = original_image.resize((new_width, new_height), Image.ANTIALIAS)
